In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
warriner14 = pd.read_csv('/Users/sven/workspace/X-ANEW/Ratings_Warriner_et_al.csv',
                 sep=',',
                 index_col=0,
                 keep_default_na=False)  # otherwise "null" gets interpreted as NaN

/Users/sven/anaconda/envs/pytorch/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
print(warriner14.shape)
warriner14.head()

(13915, 64)


,Word,V.Mean.Sum,V.SD.Sum,V.Rat.Sum,A.Mean.Sum,A.SD.Sum,A.Rat.Sum,D.Mean.Sum,D.SD.Sum,D.Rat.Sum,...,A.Rat.L,A.Mean.H,A.SD.H,A.Rat.H,D.Mean.L,D.SD.L,D.Rat.L,D.Mean.H,D.SD.H,D.Rat.H
1,aardvark,6.26,2.21,19,2.41,1.40,22,4.27,1.75,15,...,11,2.55,1.29,11,4.12,1.64,8,4.43,1.99,7
2,abalone,5.30,1.59,20,2.65,1.90,20,4.95,1.79,22,...,12,2.38,1.92,8,5.55,2.21,11,4.36,1.03,11
3,abandon,2.84,1.54,19,3.73,2.43,22,3.32,2.50,22,...,11,3.82,2.14,11,2.77,2.09,13,4.11,2.93,9
4,abandonment,2.63,1.74,19,4.95,2.64,21,2.64,1.81,28,...,14,5.29,2.63,7,2.31,1.45,16,3.08,2.19,12
5,abbey,5.85,1.69,20,2.20,1.70,20,5.00,2.02,25,...,9,2.55,1.92,11,4.83,2.18,18,5.43,1.62,7


In [4]:
# Checking the word "null" which can cause problems when reading with pandas
warriner14.loc[8290]

Word          null
V.Mean.Sum    4.32
V.SD.Sum      1.06
V.Rat.Sum       19
A.Mean.Sum    3.05
              ... 
D.SD.L        3.14
D.Rat.L         13
D.Mean.H       5.1
D.SD.H        1.97
D.Rat.H         10
Name: 8290, Length: 64, dtype: object

In [5]:
# Filtering atypical words from the warriner ratings
for i in warriner14.index:
    word = warriner14.loc[i, 'Word']
    if ((" " in word)  # multi-word expressions
        or any(x.isupper() for x in word)  # words with capital letters (typically proper nouns such as "Adidas")
        or "'" in word):  # words with apostrophe
        warriner14.drop(i, inplace=True)
    else:
        pass
        

In [6]:
len(warriner14)

13791

In [7]:
anew10 = pd.read_csv('~/julie/resources/ANEW2010/ANEW 2010/ANEW2010All.txt', sep = '\t')
print(len(anew10))
anew10.head()

2476


,Word,Wdnum,ValMn,ValSD,AroMn,AroSD,DomMn,DomSD
0,abduction,621,2.76,2.06,5.53,2.43,3.49,2.38
1,able,1041,6.74,2.00,4.30,2.17,6.83,2.04
2,abortion,622,3.50,2.30,5.39,2.80,4.59,2.54
3,absent,1042,3.69,1.72,4.73,1.76,4.35,1.87
4,absurd,623,4.26,1.82,4.36,2.20,4.73,1.72


In [8]:
anew99 = pd.read_csv('~/julie/resources/ANEW1999/ANEW99_repaired.tsv', sep='\t')
print(len(anew99))
anew99.head()

1034


,id,word,valence,arousal,dominance
0,621,abduction,2.76,5.53,3.49
1,622,abortion,3.50,5.39,4.59
2,623,absurd,4.26,4.36,4.73
3,624,abundance,6.59,5.51,5.80
4,1,abuse,1.80,6.83,3.69


In [9]:
WORDS = {99: set(anew99.word),
         10: set(anew10.Word),
         14: set(warriner14.Word)}

In [10]:
for key, words in WORDS.items():
    print("Words in {}: {}".format(key, len(words)))

Words in 99: 1034
Words in 10: 2471
Words in 14: 13791


In [11]:
for k1,v1 in WORDS.items():
    for k2, v2 in WORDS.items():
        if not k1==k2:
            print('Overlap betwenn {} and {}: {}'.format(k1, k2, len(v1.intersection(v2))))

Overlap betwenn 99 and 10: 1034
Overlap betwenn 99 and 14: 1032
Overlap betwenn 10 and 99: 1034
Overlap betwenn 10 and 14: 2328
Overlap betwenn 14 and 99: 1032
Overlap betwenn 14 and 10: 2328


Test set is anew99 $\cap$ warriner. Dev set is anew10 $\cap$ warriner - test set. Train set is warriner - train - dev.

In [14]:
TEST = WORDS[99].intersection(WORDS[14])
DEV = WORDS[10].intersection(WORDS[14]).difference(TEST)
TRAIN = WORDS[14].difference(TEST).difference(DEV)

In [15]:
print(len(TRAIN), len(DEV), len(TEST), sep='\n')

11463
1296
1032


In [16]:
# Check for overlap (there should be none)
print(len(TRAIN.intersection(DEV)),
      len(TRAIN.intersection(TEST)),
      len(DEV.intersection(TEST)),
      sep = '\n'
     )

0
0
0


In [40]:
splits = {'train': TRAIN, 
          'dev': DEV,
          'test': TEST}

PATH_CROSSLINGUAL_SPLIT = Path('.') / 'splits' / 'crosslingual'
if not PATH_CROSSLINGUAL_SPLIT.exists():
    PATH_CROSSLINGUAL_SPLIT.mkdir(parents=True)

for split, words in splits.items():
    words = sorted(list(words))
    words = '\n'.join(words)
    file_path = PATH_CROSSLINGUAL_SPLIT / (split + '.txt')
    with open(file_path, 'w') as f:
        f.write(words)
        

---